In [1]:
# General imports
import numpy as np
import torch

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from sklearn.linear_model import ARDRegression

import seaborn as sns
sns.set()

# Making data

In [79]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)

y = dataset.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1)) # observations
X = dataset.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), poly_order=2, deriv_order=3) # covariates

print(y.shape, X.shape)

(5000, 1) (5000, 12)


In [80]:
y += np.std(y) * 0.5 * np.random.randn(*y.shape)

In [81]:
1 / (np.std(y) * 0.5)**2

125.23979797653257

In [82]:
X = X / np.linalg.norm(X, axis=0, keepdims=True)

# Own implementation

In [69]:
def robust_SBL(X, y, alpha_, beta_, lambda_):
    N, M = X.shape[0], X.shape[1]
    
    gram = X.T @ X
    sigma = torch.inverse(torch.diag(1/alpha_) + gram)
    mn = sigma @ X.T @ y
    
    p = - N * torch.log(beta_) - torch.logdet(torch.eye(M) -  gram @ sigma) + beta_ * (y.T @ y - y.T @ X @ mn)
    p += -2 * N * torch.log(lambda_/2) + lambda_ * torch.sum(alpha_) - torch.log(lambda_) + torch.log(beta_)
    return p, mn

In [70]:
# Now let's optimize
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

a = torch.nn.Parameter(torch.zeros(12, dtype=torch.float32))
b = torch.nn.Parameter(-torch.log(torch.var(y)))
l = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32))

optimizer = torch.optim.Adam([a, b, l], lr=1e-2)
max_epochs=1e4

<ipython-input-70-b8c19319fde4>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
<ipython-input-70-b8c19319fde4>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


In [71]:
for epoch in torch.arange(max_epochs):
    alpha_ = torch.exp(a).clamp(max=1e8)
    beta_ = torch.exp(b).clamp(max=1e8)
    lambda_ = torch.exp(l).clamp(max=1e8)
    loss = robust_SBL(X, y, alpha_, beta_, lambda_)[0]
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss)

tensor([[-7985.3540]], grad_fn=<AddBackward0>)
tensor([[-98543.9062]], grad_fn=<AddBackward0>)
tensor([[-189124.5781]], grad_fn=<AddBackward0>)
tensor([[-189376.6406]], grad_fn=<AddBackward0>)
tensor([[-189402.6562]], grad_fn=<AddBackward0>)
tensor([[-189411.7188]], grad_fn=<AddBackward0>)
tensor([[-189415.8906]], grad_fn=<AddBackward0>)
tensor([[-189418.0781]], grad_fn=<AddBackward0>)
tensor([[-189419.2969]], grad_fn=<AddBackward0>)
tensor([[-189420.]], grad_fn=<AddBackward0>)


In [72]:
alpha_

tensor([4.3890e-11, 1.5725e-09, 1.9485e-09, 7.0646e-11, 2.5093e-11, 1.3985e-09,
        1.0690e-10, 1.1384e-10, 1.4369e-10, 7.4551e-10, 9.2023e-12, 4.3529e-11],
       grad_fn=<ClampBackward>)

In [73]:
mn = robust_SBL(X, y, alpha_, beta_, lambda_)[1]
print(mn)

tensor([[ 5.2248e-12],
        [-1.4698e-08],
        [ 9.4517e-09],
        [ 4.8987e-10],
        [-6.9351e-11],
        [-1.2400e-08],
        [ 2.9541e-10],
        [ 8.5535e-10],
        [-6.1102e-10],
        [-5.6079e-09],
        [ 2.0429e-11],
        [ 2.9408e-10]], grad_fn=<MmBackward>)


In [74]:
mn

tensor([[ 5.2248e-12],
        [-1.4698e-08],
        [ 9.4517e-09],
        [ 4.8987e-10],
        [-6.9351e-11],
        [-1.2400e-08],
        [ 2.9541e-10],
        [ 8.5535e-10],
        [-6.1102e-10],
        [-5.6079e-09],
        [ 2.0429e-11],
        [ 2.9408e-10]], grad_fn=<MmBackward>)

In [76]:
lambda_

tensor([100000000.], grad_fn=<ClampBackward>)

In [33]:
beta_

tensor(155.3127, grad_fn=<ClampBackward>)

In [77]:
a

Parameter containing:
tensor([-23.8498, -20.2711, -20.0567, -23.3738, -24.4089, -20.3884, -22.9597,
        -22.8968, -22.6639, -21.0175, -25.4121, -23.8581], requires_grad=True)